In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import cleaning
from datetime import timedelta
%load_ext autoreload
%autoreload 2

path = "../data/mimic-iv-clinical-database-demo-2.2/hosp/"
path2 = "../data/mimic-iv-clinical-database-demo-2.2/icu/"

In [2]:
## reading files and creating dataframes for tables
files = ["admissions", "patients", "labevents", "d_labitems", "prescriptions","pharmacy","transfers", "diagnoses_icd","d_icd_diagnoses","procedures_icd","d_icd_procedures"]
dfs = {}

for name in files:
    dfs[name] = pd.read_csv(path + f"{name}.csv.gz")

admissions = dfs["admissions"]
patients = dfs["patients"]
labs = dfs["labevents"]
d_labitems = dfs["d_labitems"]
prescriptions = dfs["prescriptions"]
pharmacy = dfs["pharmacy"]
transfers=dfs["transfers"]
diagnoses = dfs["diagnoses_icd"]
d_diagnoses = dfs["d_icd_diagnoses"]
procedures = dfs["procedures_icd"]
d_procedures = dfs["d_icd_procedures"]

files2 = ["icustays", "inputevents", "outputevents", "procedureevents","chartevents", "datetimeevents", "d_items"]
dfs2 = {}

for name in files2:
    dfs2[name] = pd.read_csv(path2 + f"{name}.csv.gz")
icustays = dfs2["icustays"]
inputevents = dfs2["inputevents"]
outputevents = dfs2["outputevents"]
procedureevents = dfs2["procedureevents"]
chartevents = dfs2["chartevents"]
datetimeevents = dfs2["datetimeevents"]
d_items = dfs2["d_items"]

In [3]:
cleaned = cleaning.clean("Urinary Tract Infection",4,4)
cleaned

,hadm_id,subject_id,stay_id,anchor_age,gender,race,admission_type,admission_location,admittime,dischtime,...,wbc_max,hemoglobin_min,ast_max,alt_max,bilirubin_max,inr_max,creatinine_admission_max,bun_admission_max,antibiotics,vasoactive_meds
0,23473524,10002428,35479615.0,80,F,WHITE,EW EMER.,EMERGENCY ROOM,2156-05-11 14:49:00,2156-05-22 14:16:00,...,NaN,NaN,NaN,NaN,NaN,NaN,0.4,12.0,"[CefePIME, Ciprofloxacin, Vancomycin]",[Norepinephrine]
1,25797028,10002428,NaN,80,F,WHITE,EU OBSERVATION,EMERGENCY ROOM,2155-07-14 19:15:00,2155-07-15 18:37:00,...,NaN,NaN,NaN,NaN,NaN,NaN,0.8,12.0,NaN,NaN
2,25177949,10032725,NaN,38,F,BLACK/AFRICAN AMERICAN,EW EMER.,EMERGENCY ROOM,2143-02-17 14:20:00,2143-03-16 17:15:00,...,NaN,NaN,NaN,NaN,NaN,NaN,0.9,21.0,NaN,NaN
3,22490490,10037928,31552399.0,78,F,HISPANIC/LATINO - CUBAN,EW EMER.,EMERGENCY ROOM,2177-07-14 16:55:00,2177-07-24 13:33:00,...,5.0,6.3,NaN,NaN,NaN,1.0,1.4,28.0,NaN,NaN
4,29802992,10037928,NaN,78,F,HISPANIC/LATINO - CUBAN,EW EMER.,EMERGENCY ROOM,2179-07-25 00:06:00,2179-07-28 15:54:00,...,NaN,NaN,NaN,NaN,NaN,NaN,1.8,47.0,NaN,NaN
5,24225421,10037928,NaN,78,F,HISPANIC/LATINO - CUBAN,EW EMER.,EMERGENCY ROOM,2178-09-28 23:05:00,2178-10-02 17:13:00,...,NaN,NaN,NaN,NaN,NaN,NaN,1.4,26.0,NaN,NaN
6,24656677,10037928,39804682.0,78,F,HISPANIC/LATINO - CUBAN,EW EMER.,EMERGENCY ROOM,2178-12-21 05:30:00,2178-12-26 18:35:00,...,NaN,NaN,NaN,NaN,NaN,NaN,1.3,20.0,NaN,NaN
7,22228639,10037928,NaN,78,F,HISPANIC/LATINO - CUBAN,EU OBSERVATION,PHYSICIAN REFERRAL,2183-08-04 04:04:00,2183-08-04 16:07:00,...,NaN,NaN,NaN,NaN,NaN,NaN,1.9,26.0,NaN,NaN
8,20291550,10008454,31959184.0,26,F,WHITE,EW EMER.,EMERGENCY ROOM,2110-11-30 06:31:00,2110-12-10 15:53:00,...,NaN,NaN,NaN,NaN,NaN,NaN,0.7,11.0,NaN,NaN
9,27984218,10020640,30849778.0,91,F,WHITE,EW EMER.,EMERGENCY ROOM,2153-02-13 00:22:00,2153-02-20 13:52:00,...,20.6,10.7,19.0,24.0,0.2,1.4,0.8,33.0,"[Piperacillin-Tazobactam, Azithromycin]",NaN


In [4]:
cleaned = cleaning.get_procedures(cleaned)
cleaned

,hadm_id,subject_id,stay_id,anchor_age,gender,race,admission_type,admission_location,admittime,dischtime,...,hemoglobin_min,ast_max,alt_max,bilirubin_max,inr_max,creatinine_admission_max,bun_admission_max,antibiotics,vasoactive_meds,vent_or_intubation
0,23473524,10002428,35479615.0,80,F,WHITE,EW EMER.,EMERGENCY ROOM,2156-05-11 14:49:00,2156-05-22 14:16:00,...,NaN,NaN,NaN,NaN,NaN,0.4,12.0,"[CefePIME, Ciprofloxacin, Vancomycin]",[Norepinephrine],1
1,25797028,10002428,NaN,80,F,WHITE,EU OBSERVATION,EMERGENCY ROOM,2155-07-14 19:15:00,2155-07-15 18:37:00,...,NaN,NaN,NaN,NaN,NaN,0.8,12.0,NaN,NaN,0
2,25177949,10032725,NaN,38,F,BLACK/AFRICAN AMERICAN,EW EMER.,EMERGENCY ROOM,2143-02-17 14:20:00,2143-03-16 17:15:00,...,NaN,NaN,NaN,NaN,NaN,0.9,21.0,NaN,NaN,0
3,22490490,10037928,31552399.0,78,F,HISPANIC/LATINO - CUBAN,EW EMER.,EMERGENCY ROOM,2177-07-14 16:55:00,2177-07-24 13:33:00,...,6.3,NaN,NaN,NaN,1.0,1.4,28.0,NaN,NaN,0
4,29802992,10037928,NaN,78,F,HISPANIC/LATINO - CUBAN,EW EMER.,EMERGENCY ROOM,2179-07-25 00:06:00,2179-07-28 15:54:00,...,NaN,NaN,NaN,NaN,NaN,1.8,47.0,NaN,NaN,0
5,24225421,10037928,NaN,78,F,HISPANIC/LATINO - CUBAN,EW EMER.,EMERGENCY ROOM,2178-09-28 23:05:00,2178-10-02 17:13:00,...,NaN,NaN,NaN,NaN,NaN,1.4,26.0,NaN,NaN,0
6,24656677,10037928,39804682.0,78,F,HISPANIC/LATINO - CUBAN,EW EMER.,EMERGENCY ROOM,2178-12-21 05:30:00,2178-12-26 18:35:00,...,NaN,NaN,NaN,NaN,NaN,1.3,20.0,NaN,NaN,0
7,22228639,10037928,NaN,78,F,HISPANIC/LATINO - CUBAN,EU OBSERVATION,PHYSICIAN REFERRAL,2183-08-04 04:04:00,2183-08-04 16:07:00,...,NaN,NaN,NaN,NaN,NaN,1.9,26.0,NaN,NaN,0
8,20291550,10008454,31959184.0,26,F,WHITE,EW EMER.,EMERGENCY ROOM,2110-11-30 06:31:00,2110-12-10 15:53:00,...,NaN,NaN,NaN,NaN,NaN,0.7,11.0,NaN,NaN,0
9,27984218,10020640,30849778.0,91,F,WHITE,EW EMER.,EMERGENCY ROOM,2153-02-13 00:22:00,2153-02-20 13:52:00,...,10.7,19.0,24.0,0.2,1.4,0.8,33.0,"[Piperacillin-Tazobactam, Azithromycin]",NaN,0


In [35]:
d_labitems[d_labitems["label"].str.contains("inr",case=False,na=False)]

,itemid,label,fluid,category
368,51675,INR(PT),Blood,Chemistry
1003,51237,INR(PT),Blood,Hematology


In [5]:
cleaned.to_csv("../draft1.csv")